In [1]:
import pandas as pd
import numpy as np

# READ THE CSV INTO DATAFRAME

df = pd.read_csv('Syngenta/Syngenta_2017/Experiment_dataset.csv')

# np_ar = np.asarray(df)
# df2 = pd.read_csv('Syngenta/Syngenta_2017/Region_dataset.csv')

In [ ]:
print(df['Planting date'][[np.random.randn]])
# print(df.columns)
# print(df.Temperature.describe())
# import random
# for i in range(0, 500):
#     print(random.choice(df['Planting date']))

In [ ]:
df.Yield.describe()

In [ ]:
# CURRENTLY NECESSARY IF: USING 174 ADDITIONAL VARIETY COLUMNS METHOD

# THIS IS A DIFFERENT APPROACH TO THE ABOVE FOUR CELLS, WHERE WE HAVE 174 ADDITIONAL FEATURE COLUMNS
# EACH WITH A 0 (IF IT IS NOT OF THAT VARIETY) OR A 1 (IF IT IS OF THAT VARIETY)

# print(df)
variety_dummies = pd.get_dummies(df.Variety)
# print(dummies)
df = pd.concat([df, variety_dummies], axis=1)


In [ ]:
# OPTIONAL VARIETY DISTRIBUTION ANALYSIS

print(variety_dummies.sum().describe())
print(np.sort(variety_dummies.sum()))
for idx, cl in enumerate(variety_dummies.sum()):
    print(variety_dummies.columns[idx], cl)

In [2]:
print(df.columns)

Index(['Experiment', 'Location', 'Variety', 'Planting date', 'Yield',
       'Check Yield', 'Yield difference', 'Year', 'Latitude', 'Longitude',
       'Temperature', 'Precipitation', 'Solar Radiation', 'Soil class', 'CEC',
       'Organic matter', 'pH', 'Clay', 'Silt', 'Sand', 'PI', 'Area'],
      dtype='object')


In [ ]:
# GOAL OF THIS MODULE:
# Encode the planting date as a season
# NEW GOAL:
# GET DUMMIES FOR SEASONS

# remove the dates that are "."
df = df[~df['Planting date'].str.match("\.")]
plant_date = df['Planting date'].apply(lambda dt: pd.to_datetime(dt))
plant_months = plant_date.apply(lambda dt: dt.month)
season = plant_date.rename("Season")
season = pd.to_datetime(season)
season = season.apply(lambda dt: (dt.month%12 + 3)//3)
# df['Plant date'] = pd.to_datetime(df['Plant date'])
df = pd.concat([df, season], axis=1)

# plant_date = pd.to_datetime(df['Planting date'], infer_datetime_format=True)
# df = df['Planting date'].apply(lambda dt: (dt.month%12 + 3)//3)
# pd.get_dummies(df['Planting date'])


In [ ]:
# ADD MONTH OF MAY AND JUNE ONE HOT ENCODING INTO THE DATAFRAME
pd.get_dummies(plant_months).sum()
june = pd.get_dummies(plant_months).loc[:,6]
june = june.rename("June")
may = pd.get_dummies(plant_months).loc[:,5]
may = may.rename("May")
df = pd.concat([df, may], axis=1)
df = pd.concat([df, june], axis=1)

In [ ]:
pd.get_dummies(df['Season']).sum()

In [ ]:
print(df.columns)
print(df.isnull().sum())

In [ ]:
# LATITUDE AND LONGITUDE CLUSTERING INTO FEATURES

from sklearn.cluster import KMeans

latlong = df.loc[:, ['Latitude', 'Longitude']]

kmeans = KMeans(n_clusters=4, random_state=0).fit(latlong)
kmeans.labels_.shape
lat_long_dummies = pd.get_dummies(kmeans.labels_)
lat_long_dummies = lat_long_dummies.rename(index=int, columns={0: "Loc Clust 0",
                                                               1: "Loc Clust 1",
                                                               2: "Loc Clust 2",
                                                               3: "Loc Clust 3"})
df = pd.concat([df, lat_long_dummies], axis = 1)

In [ ]:
np.asarray(df.iloc[:, df.columns.str.match('V\d\d\d\d\d\d')].columns)

In [ ]:
#THIS IS A VISUALIZATION FOR LATITUDE AND LONGITUDE CLUSTERING

cent = kmeans.cluster_centers_
clust_labels = kmeans.labels_
means = pd.DataFrame(clust_labels)

import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(111)
scatter = ax.scatter(df['Latitude'],df['Longitude'],
                     c=means[0], s=50)
ax.set_title('K-Means Clustering')
ax.set_xlabel('Latitude')
ax.set_ylabel('Longitude')
plt.colorbar(scatter)

In [ ]:
# DROP ALL THE CELLS THAT ARE NOT USABLE SUCH AS THE ONES THAT ARE STRINGS OR DATES

# set if want to drop some columns specifically
should_drop = 1
columns_to_drop = ['Experiment', 'Location',
                   'Check Yield', 'Yield difference', 'Latitude',
                   'Longitude', 'Variety', 'PI', 'Planting date']

# set if want to keep some columns specifically
should_keep = 0
# columns_to_keep = ['Loc Clust 0', 'Loc Clust 1', 'Loc Clust 2', 'Loc Clust 3']
columns_to_keep_top = ['Silt', 'Precipitation', 'Temperature', 'Solar Radiation', 'Organic matter']
columns_VARIETIES_ONLY = np.asarray(df.iloc[:, df.columns.str.match('V\d\d\d\d\d\d')].columns)

#set the below variable to whatever columns you want to keep
columns_to_keep = columns_to_keep_top

MUST_HAVE_COLUMNS = ['Yield']
# print(columns_to_keep)

df = df.drop(columns_to_drop, axis=1) if should_drop else df
df = df.loc[:, np.concatenate((columns_to_keep, MUST_HAVE_COLUMNS))] if should_keep else df
df['YieldBucket'] = pd.Series(pd.qcut(df.Yield, q=3, labels=["high", "medium", "low"]))
print("The final dataframe has columns: ", df.columns)

In [ ]:
print(pd.get_dummies(df.YieldBucket).sum())

In [ ]:
print(pd.get_dummies(df.YieldBucket).sum())